###Initialisation cleaning

In [ ]:
import pandas as pd

In [ ]:
pap = pd.read_csv("pap (1).csv")

selog = pd.read_csv("seloger_particuliers (1).csv")

bc_neuf = pd.read_csv("250311_leboncoin_neuf.csv")

bc_ancien = pd.read_csv("leboncoin (1).csv")

In [ ]:
df_pap = pd.DataFrame(pap)

df_selog = pd.DataFrame(selog)

df_neuf = pd.DataFrame(bc_neuf)

df_lbc = pd.DataFrame(bc_ancien)

###Cleaning PAP

In [ ]:
df_pap.head(5)

,item-thumb-link href,h1,item-tags,item-tags 2,item-tags 3,item-price,item-description,item-tag
0,https://www.pap.fr/annonces/appartement-vichy-...,Vichy (03200),4 pièces,3 chambres,93 m²,287.000 €,Superbe appartement d'une superficie de 93 m2 ...,NaN
1,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin (45380),4 pièces,2 chambres,110 m²,165.000 €,Situé proche de toutes les commodités (superma...,NaN
2,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine (92600),4 pièces,3 chambres,74 m²,460.000 €,"**Bel appartement dans un immeuble récent**, a...",NaN
3,NaN,Combien emprunter en mars 2025 ?,NaN,NaN,NaN,Vérifiez votre budget,Vous avez besoin d'emprunter pour financer vot...,NaN
4,https://www.pap.fr/annonces/appartement-cannes...,Cannes (06400),3 pièces,2 chambres,82 m²,615.000 €,"PALM BEACH, POINTE CROISETTE. Beau quartier ré...",NaN


In [ ]:
df_pap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4644 entries, 0 to 4643
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item-thumb-link href  4643 non-null   object
 1   h1                    4644 non-null   object
 2   item-tags             4643 non-null   object
 3   item-tags 2           4643 non-null   object
 4   item-tags 3           4219 non-null   object
 5   item-price            4644 non-null   object
 6   item-description      4644 non-null   object
 7   item-tag              929 non-null    object
dtypes: object(8)
memory usage: 290.4+ KB


In [ ]:
# remplir les NA de la colonne surface avec la colonne 2 si c'est un studio
df_pap['item-tags 3'] = df_pap['item-tags 3'].fillna(df_pap['item-tags 2'])

In [ ]:
# Rename colonne
df_pap = df_pap.rename(columns={'item-thumb-link href': 'URL',
                                'h1': 'ville',
                                'item-tags': 'nb_pieces',
                                'item-tags 3': 'surface',
                                'item-price': 'prix'})

In [ ]:
# Drop colomns que l'on ne veut pas garder
df_pap = df_pap.drop(columns=["item-tag"])
df_pap = df_pap.drop(columns=["item-description"])
df_pap = df_pap.drop(columns=["item-tags 2"])

In [ ]:
# Creation colonne code postal
df_pap['code_postal'] = df_pap['ville'].str.extract(r'\((\d{5})\)')

In [ ]:
#Supression des caractères en trop dans les données
df_pap['ville'] = df_pap['ville'].str.replace(r'\s*\(\d{5}\)', '', regex=True)
df_pap['surface'] = df_pap['surface'].str.replace(r'\s*m²', '', regex=True)
df_pap['nb_pieces'] = df_pap['nb_pieces'].str.replace(r'\s*pièces', '', regex=True)
df_pap['nb_pieces'] = df_pap['nb_pieces'].str.replace(r'\s*pièce', '', regex=True)
df_pap['prix'] = df_pap['prix'].str.replace(r'[.€]', '', regex=True)

In [ ]:
#Drop des lignes sans url

df_pap.dropna(subset=['URL'], inplace=True)

In [ ]:
#Replacer les "," par des "." pour les floats
df_pap['surface'] = df_pap['surface'].str.replace(',','.')

In [ ]:
#Cast les colonnes dans les bons types
df_pap['surface'] = df_pap['surface'].astype(float)
df_pap['nb_pieces'] = df_pap['nb_pieces'].astype(int)
df_pap['prix'] = df_pap['prix'].astype(int)
df_pap['code_postal'] = df_pap['code_postal'].astype(str)

In [ ]:
#Creation colonne prix m2
df_pap["prix_m2"] = df_pap["prix"]/df_pap["surface"]

In [ ]:
#ajout de 0 au debut du code postal si il n'a que 4 chiffres
df_pap['code_postal'] = df_pap['code_postal'].apply(lambda x: '0' + x if len(x) == 4 else x)

In [ ]:
#Drop des lignes ou on a pas pu extraire le code postal

df_pap_cleaned = df_pap[df_pap['code_postal'].astype(str).str.match(r'^\d{5}$')]

In [ ]:
df_pap_cleaned.head()

,URL,ville,nb_pieces,surface,prix,code_postal,prix_m2
0,https://www.pap.fr/annonces/appartement-vichy-...,Vichy,4,93.0,287000,03200,3086.021505
1,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin,4,110.0,165000,45380,1500.000000
2,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine,4,74.0,460000,92600,6216.216216
4,https://www.pap.fr/annonces/appartement-cannes...,Cannes,3,82.0,615000,06400,7500.000000
5,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,1,18.0,119000,92220,6611.111111


In [ ]:
df_pap_cleaned.isnull().sum()

,0
URL,0
ville,0
nb_pieces,0
surface,0
prix,0
code_postal,0
prix_m2,0


In [ ]:
df_pap_cleaned.to_csv("pap_clean.csv")

###Cleaning SeLoger

In [ ]:
df_selog.head()

,Prix,Prix_m2,Type_appart,Nb-pieces,Surface,Surface_ou_etage,Etage,Localisation,Titre_annonce,URL
0,105 000 €,10 500 € le m²,Studio,1 pièce,10 m²,Étage 6/6,Ascenseur,Baumettes à Nice (06000),Vente entre particuliers Appartement - 167 Pro...,https://www.seloger.com/annonces/achat/apparte...
1,268 000 €,3 045 € le m²,Appartement,6 pièces,5 chambres,88 m²,Étage 12/–,Prefecture à Bobigny (93000),Vente entre particuliers Appartement - AVENUE ...,https://www.seloger.com/annonces/achat/apparte...
2,169 000 €,3 930 € le m²,Appartement,2 pièces,1 chambre,43 m²,Étage 2/2,Les Rayols-Saint Lazare-Les Bousquets à Cuers ...,Vente entre particuliers Appartement - Beau T2...,https://www.seloger.com/annonces/achat/apparte...
3,155 000 €,3 690 € le m²,Appartement,2 pièces,1 chambre,42 m²,Étage 1/1,Ferrandière Maisons-Neuves à Villeurbanne (69100),Vente entre particuliers Appartement - Apparte...,https://www.seloger.com/annonces/achat/apparte...
4,165 000 €,2 538 € le m²,Appartement,4 pièces,3 chambres,65 m²,Étage 4/4,Lénine à Le Blanc-Mesnil (93150),Vente entre particuliers Appartement - LE BLAN...,https://www.seloger.com/annonces/achat/apparte...


In [ ]:
df_selog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Prix              4046 non-null   object
 1   Prix_m2           4046 non-null   object
 2   Type_appart       4046 non-null   object
 3   Nb-pieces         4046 non-null   object
 4   Surface           4046 non-null   object
 5   Surface_ou_etage  4046 non-null   object
 6   Etage             3919 non-null   object
 7   Localisation      4046 non-null   object
 8   Titre_annonce     4046 non-null   object
 9   URL               4046 non-null   object
dtypes: object(10)
memory usage: 316.2+ KB


In [ ]:
# Drop colomns que l'on ne veut pas garder
df_selog = df_selog.drop(columns=["Etage"])
df_selog = df_selog.drop(columns=["Titre_annonce"])

In [ ]:
# Rename colonne
df_selog = df_selog.rename(columns={'Localisation': 'ville',
                                    "Nb-pieces": "nb_pieces",
                                    "Surface_ou_etage": "surface",
                                    "Surface": "nb_chambre",
                                    "Prix": "prix",
                                    "Prix_m2": "prix_m2",
                                    "Type_appart": "type_appart"})

In [ ]:
# Creation colonne code postal
df_selog['code_postal'] = df_selog['ville'].str.extract(r'\((\d{5})\)')

In [ ]:
# remplir les surfaces avec la colonne nb_chambre si c'est un studio
df_selog.loc[df_selog['type_appart'] == 'Studio', 'surface'] = df_selog['nb_chambre']

In [ ]:
#Supression des caractères en trop dans les données en convertissant en str
df_selog['ville'] = df_selog['ville'].str.replace(r'\s*\(\d{5}\)', '', regex=True)

df_selog["surface"] = df_selog["surface"].str.replace(r'\s*m²', '', regex=True)

df_selog['nb_pieces'] = df_selog['nb_pieces'].str.replace(r'\s*pièces', '', regex=True)
df_selog['nb_pieces'] = df_selog['nb_pieces'].str.replace(r'\s*pièce', '', regex=True)

df_selog['prix'] = df_selog['prix'].str.replace(r'[.€]', '', regex=True)

df_selog['prix_m2'] = df_selog['prix_m2'].str.replace(r'€ le m²', '', regex=True).str.strip()

In [ ]:
df_selog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prix         4046 non-null   object
 1   prix_m2      4046 non-null   object
 2   type_appart  4046 non-null   object
 3   nb_pieces    4046 non-null   object
 4   nb_chambre   4046 non-null   object
 5   surface      4046 non-null   object
 6   ville        4046 non-null   object
 7   URL          4046 non-null   object
 8   code_postal  4046 non-null   object
dtypes: object(9)
memory usage: 284.6+ KB


In [ ]:
#Drop la derniere colonne inutile
df_selog = df_selog.drop(columns=["nb_chambre"])

In [ ]:
#Cast les colonnes dans les bons type

df_selog['surface'] = pd.to_numeric(df_selog['surface'], errors='coerce').astype(float)
df_selog['nb_pieces'] = pd.to_numeric(df_selog['nb_pieces'], errors='coerce').astype(int)
df_selog['prix'] = df_selog['prix'].str.replace(' ', '').str.strip()
df_selog['prix'] = pd.to_numeric(df_selog['prix'], errors='coerce').astype(float)
df_selog['code_postal'] = df_selog['code_postal'].astype(str)

In [ ]:
df_selog['code_postal'] = df_selog['code_postal'].apply(lambda x: '0' + x if len(x) == 4 else x)

In [ ]:
df_selog.head()

,prix,prix_m2,type_appart,nb_pieces,surface,ville,URL,code_postal
0,105000.0,10 500,Studio,1,10.0,Baumettes à Nice,https://www.seloger.com/annonces/achat/apparte...,06000
1,268000.0,3 045,Appartement,6,88.0,Prefecture à Bobigny,https://www.seloger.com/annonces/achat/apparte...,93000
2,169000.0,3 930,Appartement,2,43.0,Les Rayols-Saint Lazare-Les Bousquets à Cuers,https://www.seloger.com/annonces/achat/apparte...,83390
3,155000.0,3 690,Appartement,2,42.0,Ferrandière Maisons-Neuves à Villeurbanne,https://www.seloger.com/annonces/achat/apparte...,69100
4,165000.0,2 538,Appartement,4,65.0,Lénine à Le Blanc-Mesnil,https://www.seloger.com/annonces/achat/apparte...,93150


In [ ]:
df_selog.isnull().sum()

,0
prix,0
prix_m2,0
type_appart,0
nb_pieces,0
surface,8
ville,0
URL,0
code_postal,0


In [ ]:
df_selog.to_csv("selog_clean.csv")

### Cleaning BC_neuf

In [ ]:
#regarder les info de df_neuf
df_neuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   lien_URL          584 non-null    object
 1   caractéristiques  584 non-null    object
 2   prix              457 non-null    object
 3   neuf_bool         584 non-null    object
 4   localisation      584 non-null    object
 5   date              552 non-null    object
 6   prix_baissé       122 non-null    object
 7   état              122 non-null    object
dtypes: object(8)
memory usage: 36.6+ KB


In [ ]:
df_neuf.head(5)

,lien_URL,caractéristiques,prix,neuf_bool,localisation,date,prix_baissé,état
0,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 2 pièces, 37 mètres carrés",191 000 €,Neuf,Granville 50400,NaN,NaN,NaN
1,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 2 pièces, 41 mètres carrés",141 300 €,Neuf,Douai 59500,NaN,NaN,NaN
2,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 4 pièces, 86 mètres carrés",275 000 €,Neuf,Miramas 13140,aujourd’hui à 09:03,NaN,NaN
3,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 4 pièces, 104 mètres carrés",368 500 €,Neuf,Riedisheim 68400,aujourd’hui à 09:01,NaN,NaN
4,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 4 pièces, 81 mètres carrés",339 500 €,Neuf,Saint-Laurent-de-Mure 69720,aujourd’hui à 09:01,NaN,NaN


In [ ]:
df_neuf.describe()

,lien_URL,caractéristiques,prix,neuf_bool,localisation,date,prix_baissé,état
count,584,584,457,584,584,552,122,122
unique,584,145,323,2,404,128,110,1
top,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 3 pièces, 62 mètres carrés",215 000 €,Neuf,Strasbourg 67000,hier à 16:24,251 000 €,Baisse de prix
freq,1,26,7,534,9,23,3,122


In [ ]:
#création d'une colone "code_postal" vide
df_neuf["code_postal"] = [None] * len(df_neuf)

In [ ]:
# séparer le code postal de la suite de caractère pour les indiquer dans la colonne "code_postal"
df_neuf['code_postal'] = df_neuf['localisation'].str.extract(r'(\d{5})')

In [ ]:
#supprimer les caractères équivalent au code postale et virgule de la colonne "localisation"
df_neuf['localisation'] = df_neuf['localisation'].str.replace(r',?\s*\d{5}$', '', regex=True)

In [ ]:
# Fonction pour extraire les informations, avec gestion des erreurs
def extract_info(description):
    parts = description.split(',')

    # Vérifie que la chaîne a bien 3 parties
    if len(parts) == 3:
        _type = parts[0].strip()  # "Appartement"
        nb_pieces = int(parts[1].strip().split()[0])  # "2" pièces
        surface_m2 = int(parts[2].strip().split()[0])  # "37" mètres carrés
    else:
        # Si le format n'est pas correct, retourne des valeurs manquantes ou par défaut
        _type = nb_pieces = surface_m2 = None

    return _type, nb_pieces, surface_m2

# Appliquer la fonction à la colonne 'caractéristiques' et créer les nouvelles colonnes
df_neuf[['_type', 'nb_pieces', 'surface_m2']] = df_neuf['caractéristiques'].apply(lambda x: pd.Series(extract_info(x)))

In [ ]:
df_neuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   lien_URL          584 non-null    object 
 1   caractéristiques  584 non-null    object 
 2   prix              457 non-null    object 
 3   neuf_bool         584 non-null    object 
 4   localisation      584 non-null    object 
 5   date              552 non-null    object 
 6   prix_baissé       122 non-null    object 
 7   état              122 non-null    object 
 8   code_postal       584 non-null    object 
 9   _type             579 non-null    object 
 10  nb_pieces         579 non-null    float64
 11  surface_m2        579 non-null    float64
dtypes: float64(2), object(10)
memory usage: 54.9+ KB


In [ ]:
#supprimmer la colonne "caractéristiques"
df_neuf = df_neuf.drop("caractéristiques",axis=1)

In [ ]:
#suprimmer les caractères " €" de la colonne prix
df_neuf["prix"] = df_neuf["prix"].str[:-2]

In [ ]:
#supprimmer la colonne "date"
df_neuf = df_neuf.drop("date",axis=1)

In [ ]:
#remplacer les "neuf" en valeur 1 boolean
df_neuf["neuf_bool"] = df_neuf["neuf_bool"].replace(to_replace="Neuf", value=1)

In [ ]:
#suprimmer les caractères " €" de la colonne prix_baissé
df_neuf["prix_baissé"] = df_neuf["prix_baissé"].str[:-2]

In [ ]:
##remplacer les "Baisse de prix" en valeur 1 boolean
df_neuf["état"] = df_neuf["état"].replace(to_replace="Baisse de prix", value=1)

<ipython-input-1509-877285a60a61>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_neuf["état"] = df_neuf["état"].replace(to_replace="Baisse de prix", value=1)


In [ ]:
#renommer la colonne "état" en "baisse_prix"
df_neuf = df_neuf.rename(columns={"état": "baisse_prix"})

In [ ]:
#remplacer les valeurs nulles NaN par 0 dans la colonne "baisse_prix"
df_neuf["baisse_prix"].fillna(0, inplace=True)

<ipython-input-1511-c87b501002ec>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_neuf["baisse_prix"].fillna(0, inplace=True)


In [ ]:
#changer le type de "baisse_prix" de string en integer (colonne boolean)
df_neuf["baisse_prix"] = df_neuf["baisse_prix"].astype(int)

In [ ]:
#remplacer les valeurs "Avant-première" par un 1 en les considérant comme la valeur "neuf"
df_neuf["neuf_bool"] = df_neuf["neuf_bool"].replace(to_replace="Avant-première", value=1)

<ipython-input-1513-79fde9c4ca77>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_neuf["neuf_bool"] = df_neuf["neuf_bool"].replace(to_replace="Avant-première", value=1)


In [ ]:
#changer le type de "neuf_bool" de string en integer (colonne boolean)
df_neuf["neuf_bool"] = df_neuf["neuf_bool"].astype(int)

In [ ]:
#remplacer les valeurs nulles de la colonne prix avec les valeurs non nulles de la colonne prix baissé (correspondance des valeurs)
df_neuf["prix"]=df_neuf["prix"].fillna(df_neuf["prix_baissé"])

In [ ]:
# Filtrer les lignes où la colonne 'prix' contient un '\'
df_avec_backslash = df_neuf[df_neuf['prix'].str.contains(r'\\', regex=True, na=False)]

print(df_avec_backslash)

Empty DataFrame
Columns: [lien_URL, prix, neuf_bool, localisation, prix_baissé, baisse_prix, code_postal, _type, nb_pieces, surface_m2]
Index: []


In [ ]:
#il y a 5 valeurs de prix qui sont nulles ou inexistantes
df_neuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lien_URL      584 non-null    object 
 1   prix          579 non-null    object 
 2   neuf_bool     584 non-null    int64  
 3   localisation  584 non-null    object 
 4   prix_baissé   122 non-null    object 
 5   baisse_prix   584 non-null    int64  
 6   code_postal   584 non-null    object 
 7   _type         579 non-null    object 
 8   nb_pieces     579 non-null    float64
 9   surface_m2    579 non-null    float64
dtypes: float64(2), int64(2), object(6)
memory usage: 45.8+ KB


In [ ]:
# Remplacer les occurrences de '\u202f' par une chaîne vide, en ignorant les valeurs NaN
df_neuf['prix'] = df_neuf['prix'].fillna('').str.replace(r'\u202f', '', regex=True)

# Supprimer les lignes où la colonne 'prix' est vide (chaîne vide)
df_neuf = df_neuf[df_neuf['prix'] != '']

# Convertir la colonne 'prix' en entier
df_neuf['prix'] = df_neuf['prix'].astype(int)

In [ ]:
#supprimmer la colonne "prix_baissé"
df_neuf = df_neuf.drop("prix_baissé",axis=1)

In [ ]:
# Fonction pour vérifier si une valeur est un entier
def is_integer(value):
    try:
        # Tente de convertir la valeur en entier
        int(value)
        return True
    except ValueError:
        # Si une exception est levée, la valeur n'est pas un entier
        return False

# Filtrer les lignes où la valeur de 'nb_pieces' n'est pas un entier
df_non_entiers = df_neuf[~df_neuf['nb_pieces'].apply(is_integer)]

In [ ]:
#changer le type de "prix" de string en float
df_neuf["prix"] = df_neuf["prix"].astype(float)

In [ ]:
#renommer la colonne "localisation" en "ville"
df_neuf = df_neuf.rename(columns={"localisation": "ville"})

In [ ]:
# Filtrer les lignes contenant au moins une valeur NaN dans les colonnes spécifiques
df_null = df_neuf[df_neuf[['_type', 'nb_pieces', 'surface_m2']].isna().any(axis=1)]

In [ ]:
# Supprimer les lignes contenant des valeurs NaN dans les colonnes spécifiques
df_neuf = df_neuf.dropna(subset=['_type', 'nb_pieces', 'surface_m2'])

In [ ]:
df_neuf['code_postal'] = df_neuf['code_postal'].apply(lambda x: '0' + x if len(x) == 4 else x)

In [ ]:
df_neuf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, 0 to 583
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lien_URL     577 non-null    object 
 1   prix         577 non-null    float64
 2   neuf_bool    577 non-null    int64  
 3   ville        577 non-null    object 
 4   baisse_prix  577 non-null    int64  
 5   code_postal  577 non-null    object 
 6   _type        577 non-null    object 
 7   nb_pieces    577 non-null    float64
 8   surface_m2   577 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 45.1+ KB


In [ ]:
df_neuf.to_csv("leboncoin_neuf_clean.csv", index=False)

### Cleaning BC_ancien

In [ ]:
#regarder les info de df_neuf
df_lbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   lien_URL          508 non-null    object
 1   caractéristiques  508 non-null    object
 2   prix_baissé       125 non-null    object
 3   état              125 non-null    object
 4   prix_2            503 non-null    object
 5   pro_bool          469 non-null    object
 6   localisation      508 non-null    object
 7   prix              383 non-null    object
 8   date              472 non-null    object
dtypes: object(9)
memory usage: 35.8+ KB


In [ ]:
df_lbc.head(3)

,lien_URL,caractéristiques,prix_baissé,état,prix_2,pro_bool,localisation,prix,date
0,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 3 pièces, 61 mètres carrés",209 000 €,Baisse de prix,3 426 €/m²,Pro,Lille 59260 Hellemmes-Lille,NaN,NaN
1,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 2 pièces, 42 mètres carrés",190 365 €,Baisse de prix,4 533 €/m²,Pro,Rennes 35200,NaN,NaN
2,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 4 pièces, 68 mètres carrés",NaN,NaN,4 191 €/m²,NaN,Volmerange-les-Mines 57330,285 000 €,aujourd’hui à 17:25


In [ ]:
df_lbc.describe()

,lien_URL,caractéristiques,prix_baissé,état,prix_2,pro_bool,localisation,prix,date
count,508,508,125,125,503,469,508,383,472
unique,506,210,104,1,473,1,381,258,15
top,https://www.leboncoin.fr/ad/ventes_immobiliere...,"Appartement, 2 pièces, 44 mètres carrés",229 000 €,Baisse de prix,3 000 €/m²,Pro,Toulon 83000,195 000 €,aujourd’hui à 17:21
freq,2,13,3,125,4,469,7,6,409


In [ ]:
# Extraire uniquement les chiffres de la colonne 'localisation'
df_lbc['code_postal'] = df_lbc['localisation'].str.extract(r'(\d+)')

In [ ]:
# Supprimer tous les chiffres de la colonne 'localisation'
df_lbc['localisation'] = df_lbc['localisation'].str.replace(r'\d+', '', regex=True).str.strip()

In [ ]:
# Fonction pour extraire les informations, avec gestion des erreurs
def extract_info(description):
    parts = description.split(',')

    # Vérifie que la chaîne a bien 3 parties
    if len(parts) == 3:
        _type = parts[0].strip()  # "Appartement"
        nb_pieces = int(parts[1].strip().split()[0])  # "2" pièces
        surface_m2 = int(parts[2].strip().split()[0])  # "37" mètres carrés
    else:
        # Si le format n'est pas correct, retourne des valeurs manquantes ou par défaut
        _type = nb_pieces = surface_m2 = None

    return _type, nb_pieces, surface_m2

# Appliquer la fonction à la colonne 'caractéristiques' et créer les nouvelles colonnes
df_lbc[['_type', 'nb_pieces', 'surface_m2']] = df_lbc['caractéristiques'].apply(lambda x: pd.Series(extract_info(x)))

In [ ]:
#supprimmer la colonne "caractéristiques"
df_lbc = df_lbc.drop("caractéristiques",axis=1)

In [ ]:
#suprimmer les caractères " €" de la colonne prix
df_lbc["prix"] = df_lbc["prix"].str[:-2]

In [ ]:
#supprimmer la colonne "date"
df_lbc = df_lbc.drop("date",axis=1)

In [ ]:
#renommer la colonne prix_2 en "prix_m2"
df_lbc = df_lbc.rename(columns={"prix_2": "prix_m2"})

In [ ]:
#renommer la colonne "localisation" en "ville"
df_lbc = df_lbc.rename(columns={"localisation": "ville"})

In [ ]:
#remplacer les "pro" en valeur 1 boolean
df_lbc["pro_bool"] = df_lbc["pro_bool"].replace(to_replace="Pro", value=1)

<ipython-input-1539-2aeb05253ebf>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_lbc["pro_bool"] = df_lbc["pro_bool"].replace(to_replace="Pro", value=1)


In [ ]:
#remplacer les "NaN" de la colonne pro_bool en valeur 1 boolean
df_lbc["pro_bool"] = df_lbc["pro_bool"].fillna(0)

In [ ]:
#suprimmer les caractères " €" de la colonne prix_baissé
df_lbc["prix_baissé"] = df_lbc["prix_baissé"].str[:-2]

In [ ]:
##remplacer les "Baisse de prix" en valeur 1 boolean
df_lbc["état"] = df_lbc["état"].replace(to_replace="Baisse de prix", value=1)

<ipython-input-1542-8dc621e67a41>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_lbc["état"] = df_lbc["état"].replace(to_replace="Baisse de prix", value=1)


In [ ]:
#renommer la colonne "état" en "baisse_prix"
df_lbc = df_lbc.rename(columns={"état": "baisse_prix"})

In [ ]:
#remplacer les valeurs nulles de la colonne prix avec les valeurs non nulles de la colonne prix baissé (correspondance des valeurs)
df_lbc["prix"]=df_lbc["prix"].fillna(df_lbc["prix_baissé"])

In [ ]:
#remplacer les valeurs nulles NaN par 0 dans la colonne "baisse_prix"
df_lbc["baisse_prix"].fillna(0, inplace=True)

<ipython-input-1545-a9679f5dc77e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_lbc["baisse_prix"].fillna(0, inplace=True)


In [ ]:
#remplacer les valeurs "Baisse de prix" par un 1 en les considérant comme telle
df_lbc["baisse_prix"] = df_lbc["baisse_prix"].replace(to_replace="Baisse de prix", value=1)

In [ ]:
#changer le type de "baisse_prix" de string en integer (colonne boolean)
df_lbc["baisse_prix"] = df_lbc["baisse_prix"].astype(int)

In [ ]:
#changer le type de "pro_bool" de string en integer (colonne boolean)
df_lbc["pro_bool"] = df_lbc["pro_bool"].astype(int)

In [ ]:
#supprimmer la colonne "prix_baissé"
df_lbc = df_lbc.drop("prix_baissé",axis=1)

In [ ]:
# Filtrer les lignes où la colonne 'prix' contient un '\'
df_avec_backslash = df_lbc[df_lbc['prix'].str.contains(r'\\', regex=True, na=False)]

print(df_avec_backslash)

Empty DataFrame
Columns: [lien_URL, baisse_prix, prix_m2, pro_bool, ville, prix, code_postal, _type, nb_pieces, surface_m2]
Index: []


In [ ]:
# Remplacer les occurrences de '\u202f' par une chaîne vide, en ignorant les valeurs NaN
df_lbc['prix'] = df_lbc['prix'].fillna('').str.replace(r'\u202f', '', regex=True)

# Supprimer les lignes où la colonne 'prix' est vide (chaîne vide)
df_lbc = df_lbc[df_lbc['prix'] != '']

# Convertir la colonne 'prix' en entier
df_lbc['prix'] = df_lbc['prix'].astype(int)

In [ ]:
# ne garder que les chiffres dans la colonne 'prix_m2'
df_lbc['prix_m2'] = df_lbc['prix_m2'].str.replace(r'\D+', '', regex=True)

In [ ]:
#changer le type de "prix" de string en float
df_lbc["prix"] = df_lbc["prix"].astype(float)

In [ ]:
#il y a 8 valeurs de _type/nb_piecs/surface_m2 qui sont nulles ou inexistantes
df_lbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lien_URL     508 non-null    object 
 1   baisse_prix  508 non-null    int64  
 2   prix_m2      503 non-null    object 
 3   pro_bool     508 non-null    int64  
 4   ville        508 non-null    object 
 5   prix         508 non-null    float64
 6   code_postal  508 non-null    object 
 7   _type        500 non-null    object 
 8   nb_pieces    500 non-null    float64
 9   surface_m2   500 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 39.8+ KB


In [ ]:
# Fonction pour vérifier si une valeur est un entier
def is_integer(value):
    try:
        # Tente de convertir la valeur en entier
        int(value)
        return True
    except ValueError:
        # Si une exception est levée, la valeur n'est pas un entier
        return False

# Filtrer les lignes où la valeur de 'nb_pieces' n'est pas un entier
df_non_entiers = df_lbc[~df_lbc['nb_pieces'].apply(is_integer)]

In [ ]:
# Filtrer les lignes contenant au moins une valeur NaN dans les colonnes spécifiques
df_null = df_lbc[df_lbc[['_type', 'nb_pieces', 'surface_m2']].isna().any(axis=1)]

In [ ]:
# Supprimer les lignes contenant des valeurs NaN dans les colonnes spécifiques
df_lbc = df_lbc.dropna(subset=['_type', 'nb_pieces', 'surface_m2'])

In [ ]:
#changer le type de "prix_m2" de string en float
df_lbc["prix_m2"] = df_lbc["prix_m2"].astype(float)

<ipython-input-1558-c33bc0fd9e0f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lbc["prix_m2"] = df_lbc["prix_m2"].astype(float)


In [ ]:
df_lbc['code_postal'] = df_lbc['code_postal'].apply(lambda x: '0' + x if len(x) == 4 else x)

<ipython-input-1559-3ca210451233>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lbc['code_postal'] = df_lbc['code_postal'].apply(lambda x: '0' + x if len(x) == 4 else x)


In [ ]:
df_lbc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 507
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lien_URL     500 non-null    object 
 1   baisse_prix  500 non-null    int64  
 2   prix_m2      500 non-null    float64
 3   pro_bool     500 non-null    int64  
 4   ville        500 non-null    object 
 5   prix         500 non-null    float64
 6   code_postal  500 non-null    object 
 7   _type        500 non-null    object 
 8   nb_pieces    500 non-null    float64
 9   surface_m2   500 non-null    float64
dtypes: float64(4), int64(2), object(4)
memory usage: 43.0+ KB


In [ ]:
df_lbc.to_csv("leboncoin_ancien_clean.csv", index=False)

Apres avoir cleané nous mergeons les annonces pour avoir un seul df

###Initialisation merge

In [ ]:
#open csv
pap = pd.read_csv("pap_clean.csv")

selog = pd.read_csv("selog_clean.csv")

bc_neuf = pd.read_csv("leboncoin_neuf_clean.csv")

bc_ancien = pd.read_csv("leboncoin_ancien_clean.csv")

In [ ]:
#load data frame

df_pap = pd.DataFrame(pap)

df_selog = pd.DataFrame(selog)

df_bc_neuf = pd.DataFrame(bc_neuf)

df_bc_ancien = pd.DataFrame(bc_ancien)

### Check PAP

In [ ]:
# uniformisation des colonnes
df_pap = df_pap.rename(columns={'URL': 'url'})

In [ ]:
#drop colonne inutile
df_pap = df_pap.drop(columns=["Unnamed: 0"])

In [ ]:
#drop les lignes sans code postal
df_pap = df_pap.dropna(subset=['code_postal'])

In [ ]:
#ajout colonne site source
df_pap["source"] = "pap"

In [ ]:
df_pap.head()

,url,ville,nb_pieces,surface,prix,code_postal,prix_m2,source
0,https://www.pap.fr/annonces/appartement-vichy-...,Vichy,4,93.0,287000,3200,3086.021505,pap
1,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin,4,110.0,165000,45380,1500.000000,pap
2,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine,4,74.0,460000,92600,6216.216216,pap
3,https://www.pap.fr/annonces/appartement-cannes...,Cannes,3,82.0,615000,6400,7500.000000,pap
4,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,1,18.0,119000,92220,6611.111111,pap


In [ ]:
#conversion des codes_postaux en str pour tout conserver meme les 0 et les lettres
df_pap['code_postal'] = df_pap['code_postal'].astype(str)

In [ ]:
df_pap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3854 entries, 0 to 3853
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          3854 non-null   object 
 1   ville        3854 non-null   object 
 2   nb_pieces    3854 non-null   int64  
 3   surface      3854 non-null   float64
 4   prix         3854 non-null   int64  
 5   code_postal  3854 non-null   object 
 6   prix_m2      3854 non-null   float64
 7   source       3854 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 241.0+ KB


### Check Se Loger

In [ ]:
df_selog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4046 non-null   int64  
 1   prix         4046 non-null   float64
 2   prix_m2      4046 non-null   object 
 3   type_appart  4046 non-null   object 
 4   nb_pieces    4046 non-null   int64  
 5   surface      4038 non-null   float64
 6   ville        4046 non-null   object 
 7   URL          4046 non-null   object 
 8   code_postal  4046 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 284.6+ KB


In [ ]:
# uniformisation des colonnes
df_selog = df_selog.rename(columns={'Prix': 'prix',
                                'Prix_m2': 'prix_m2',
                                'Type_appart': 'type_appart',
                                'URL': 'url'})

In [ ]:
# Drop ancien index
df_selog = df_selog.drop(columns=["Unnamed: 0"])

In [ ]:
#ajout colonne site source
df_selog["source"] = "se_loger"

In [ ]:
#conversion des codes_postaux en str pour tout conserver meme les 0 et les lettres
df_selog['code_postal'] = df_selog['code_postal'].astype(str)

In [ ]:
df_selog.head()

,prix,prix_m2,type_appart,nb_pieces,surface,ville,url,code_postal,source
0,105000.0,10 500,Studio,1,10.0,Baumettes à Nice,https://www.seloger.com/annonces/achat/apparte...,6000,se_loger
1,268000.0,3 045,Appartement,6,88.0,Prefecture à Bobigny,https://www.seloger.com/annonces/achat/apparte...,93000,se_loger
2,169000.0,3 930,Appartement,2,43.0,Les Rayols-Saint Lazare-Les Bousquets à Cuers,https://www.seloger.com/annonces/achat/apparte...,83390,se_loger
3,155000.0,3 690,Appartement,2,42.0,Ferrandière Maisons-Neuves à Villeurbanne,https://www.seloger.com/annonces/achat/apparte...,69100,se_loger
4,165000.0,2 538,Appartement,4,65.0,Lénine à Le Blanc-Mesnil,https://www.seloger.com/annonces/achat/apparte...,93150,se_loger


In [ ]:
df_selog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   prix         4046 non-null   float64
 1   prix_m2      4046 non-null   object 
 2   type_appart  4046 non-null   object 
 3   nb_pieces    4046 non-null   int64  
 4   surface      4038 non-null   float64
 5   ville        4046 non-null   object 
 6   url          4046 non-null   object 
 7   code_postal  4046 non-null   object 
 8   source       4046 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 284.6+ KB


### Check BC neuf

In [ ]:
df_bc_neuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lien_URL     577 non-null    object 
 1   prix         577 non-null    float64
 2   neuf_bool    577 non-null    int64  
 3   ville        577 non-null    object 
 4   baisse_prix  577 non-null    int64  
 5   code_postal  577 non-null    int64  
 6   _type        577 non-null    object 
 7   nb_pieces    577 non-null    float64
 8   surface_m2   577 non-null    float64
dtypes: float64(3), int64(3), object(3)
memory usage: 40.7+ KB


In [ ]:
# uniformisation des colonnes
df_bc_neuf = df_bc_neuf.rename(columns={'lien_URL': 'url',
                                '_type': 'type_appart',
                                'surface_m2': 'surface'})

In [ ]:
# Drop les colonnes non présentes dans les autres datasets pour uniformisation
df_bc_neuf = df_bc_neuf.drop(columns=["neuf_bool"])
df_bc_neuf = df_bc_neuf.drop(columns=["baisse_prix"])

In [ ]:
#ajout prix_m2
df_bc_neuf['prix_m2'] = df_bc_neuf["prix"]/df_bc_neuf["surface"]

In [ ]:
#ajout colonne site source
df_bc_neuf["source"] = "bon_coin_neuf"

In [ ]:
df_bc_neuf.head()

,url,prix,ville,code_postal,type_appart,nb_pieces,surface,prix_m2,source
0,https://www.leboncoin.fr/ad/ventes_immobiliere...,191000.0,Granville,50400,Appartement,2.0,37.0,5162.162162,bon_coin_neuf
1,https://www.leboncoin.fr/ad/ventes_immobiliere...,141300.0,Douai,59500,Appartement,2.0,41.0,3446.341463,bon_coin_neuf
2,https://www.leboncoin.fr/ad/ventes_immobiliere...,275000.0,Miramas,13140,Appartement,4.0,86.0,3197.674419,bon_coin_neuf
3,https://www.leboncoin.fr/ad/ventes_immobiliere...,368500.0,Riedisheim,68400,Appartement,4.0,104.0,3543.269231,bon_coin_neuf
4,https://www.leboncoin.fr/ad/ventes_immobiliere...,339500.0,Saint-Laurent-de-Mure,69720,Appartement,4.0,81.0,4191.358025,bon_coin_neuf


### Check BC ancien

In [ ]:
# uniformisation des colonnes
df_bc_ancien = df_bc_ancien.rename(columns={'lien_URL': 'url',
                                '_type': 'type_appart',
                                'surface_m2': 'surface'})

In [ ]:
# Drop les colonnes non présentes dans les autres datasets pour uniformisation
df_bc_ancien = df_bc_ancien.drop(columns=["pro_bool"])
df_bc_ancien = df_bc_ancien.drop(columns=["baisse_prix"])

In [ ]:
#ajout colonne site source
df_bc_ancien["source"] = "bon_coin_ancien"

In [ ]:
#conversion des codes_postaux en str pour tout conserver meme les 0 et les lettres
df_bc_ancien['code_postal'] = df_bc_ancien['code_postal'].astype(str)

In [ ]:
df_bc_ancien.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          500 non-null    object 
 1   prix_m2      500 non-null    float64
 2   ville        500 non-null    object 
 3   prix         500 non-null    float64
 4   code_postal  500 non-null    object 
 5   type_appart  500 non-null    object 
 6   nb_pieces    500 non-null    float64
 7   surface      500 non-null    float64
 8   source       500 non-null    object 
dtypes: float64(4), object(5)
memory usage: 35.3+ KB


In [ ]:
df_bc_ancien.head()

,url,prix_m2,ville,prix,code_postal,type_appart,nb_pieces,surface,source
0,https://www.leboncoin.fr/ad/ventes_immobiliere...,3426.0,Lille Hellemmes-Lille,209000.0,59260,Appartement,3.0,61.0,bon_coin_ancien
1,https://www.leboncoin.fr/ad/ventes_immobiliere...,4533.0,Rennes,190365.0,35200,Appartement,2.0,42.0,bon_coin_ancien
2,https://www.leboncoin.fr/ad/ventes_immobiliere...,4191.0,Volmerange-les-Mines,285000.0,57330,Appartement,4.0,68.0,bon_coin_ancien
3,https://www.leboncoin.fr/ad/ventes_immobiliere...,4114.0,Sallanches,325000.0,74700,Appartement,4.0,79.0,bon_coin_ancien
4,https://www.leboncoin.fr/ad/ventes_immobiliere...,2854.0,Troyes,99900.0,10000,Appartement,2.0,35.0,bon_coin_ancien


###Merge

In [ ]:
df_annonces_concat = pd.concat([df_pap, df_selog, df_bc_neuf, df_bc_ancien], ignore_index=True)

In [ ]:
df_annonces_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8977 entries, 0 to 8976
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          8977 non-null   object 
 1   ville        8977 non-null   object 
 2   nb_pieces    8977 non-null   float64
 3   surface      8969 non-null   float64
 4   prix         8977 non-null   float64
 5   code_postal  8977 non-null   object 
 6   prix_m2      8977 non-null   object 
 7   source       8977 non-null   object 
 8   type_appart  5123 non-null   object 
dtypes: float64(3), object(6)
memory usage: 631.3+ KB


In [ ]:
df_annonces_concat.isnull().sum()

,0
url,0
ville,0
nb_pieces,0
surface,8
prix,0
code_postal,0
prix_m2,0
source,0
type_appart,3854


In [ ]:
df_annonces_concat.head(10)

,url,ville,nb_pieces,surface,prix,code_postal,prix_m2,source,type_appart
0,https://www.pap.fr/annonces/appartement-vichy-...,Vichy,4.0,93.0,287000.0,3200,3086.021505,pap,NaN
1,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin,4.0,110.0,165000.0,45380,1500.0,pap,NaN
2,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine,4.0,74.0,460000.0,92600,6216.216216,pap,NaN
3,https://www.pap.fr/annonces/appartement-cannes...,Cannes,3.0,82.0,615000.0,6400,7500.0,pap,NaN
4,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,1.0,18.0,119000.0,92220,6611.111111,pap,NaN
5,https://www.pap.fr/annonces/appartement-troyes...,Troyes,3.0,82.0,219000.0,10000,2670.731707,pap,NaN
6,https://www.pap.fr/annonces/appartement-paris-...,Paris 3E,2.0,39.0,470000.0,75003,12051.282051,pap,NaN
7,https://www.pap.fr/annonces/appartement-marsei...,Marseille 12E,4.0,81.0,265000.0,13012,3271.604938,pap,NaN
8,https://www.pap.fr/annonces/appartement-palais...,Palaiseau,5.0,105.0,325000.0,91120,3095.238095,pap,NaN
9,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,2.0,28.0,205000.0,92220,7321.428571,pap,NaN


In [ ]:
#drop type d'appart
df_annonces_concat = df_annonces_concat.drop(columns=["type_appart"])

In [ ]:
#mettre les colonnes dans les bons formats
df_annonces_concat['nb_pieces'] = pd.to_numeric(df_annonces_concat['nb_pieces'], errors='coerce').fillna(0).astype(int)
df_annonces_concat['surface'] = pd.to_numeric(df_annonces_concat['surface'], errors='coerce').fillna(0.0).astype(float)
df_annonces_concat['prix'] = pd.to_numeric(df_annonces_concat['prix'], errors='coerce').fillna(0).astype(int)
df_annonces_concat['prix_m2'] = pd.to_numeric(df_annonces_concat['prix_m2'], errors='coerce').fillna(0.0).astype(float)

In [ ]:
df_annonces_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8977 entries, 0 to 8976
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          8977 non-null   object 
 1   ville        8977 non-null   object 
 2   nb_pieces    8977 non-null   int64  
 3   surface      8977 non-null   float64
 4   prix         8977 non-null   int64  
 5   code_postal  8977 non-null   object 
 6   prix_m2      8977 non-null   float64
 7   source       8977 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 561.2+ KB


In [ ]:
df_annonces_concat.to_csv('Concat_annonces_scaping.csv')

###Cleaning de la df prix de location au m2 par code postal

In [ ]:
indicateur = pd.read_csv("loyer_moyen_aggreg.csv")

df_loyers_m2 = pd.DataFrame(indicateur)

In [ ]:
#netoyage des données pour enlever les crochets et les '
df_loyers_m2['departement_nom'] = df_loyers_m2['departement_nom'].str.replace(r"[\[\]']", '', regex=True)
df_loyers_m2['region_nom'] = df_loyers_m2['region_nom'].str.replace(r"[\[\]']", '', regex=True)

In [ ]:
#vérification des codes postaux à 4 chiffres
nb_codes_a_quatre_chiffres = df_loyers_m2['code_postal'].str.match(r'^\d{4}$').sum()
nb_codes_a_quatre_chiffres

0

In [ ]:
df_loyers_m2.head()

,code_postal,departement,departement_nom,region,region_nom,loyer_moyen_par_m2,loyer_borne_basse_par_m2,loyer_borne_haute_par_m2,population
0,48300,48,LOZERE,76,LANGUEDOC-ROUSSILLON,7.784986,6.013423,10.081334,4.1
1,48250,48,LOZERE,76,LANGUEDOC-ROUSSILLON,7.334682,5.669581,9.490735,0.4
2,55140,55,MEUSE,44,LORRAINE,7.536209,6.087231,9.330883,5.1
3,55290,55,MEUSE,44,LORRAINE,7.243256,5.873562,8.932356,1.2
4,55130,55,MEUSE,44,LORRAINE,7.360761,5.937251,9.126532,4.5


In [ ]:
df_loyers_m2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6088 entries, 0 to 6087
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   code_postal               6088 non-null   object 
 1   departement               6088 non-null   object 
 2   departement_nom           6088 non-null   object 
 3   region                    6088 non-null   int64  
 4   region_nom                6088 non-null   object 
 5   loyer_moyen_par_m2        6088 non-null   float64
 6   loyer_borne_basse_par_m2  6088 non-null   float64
 7   loyer_borne_haute_par_m2  6088 non-null   float64
 8   population                6088 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 428.2+ KB


In [ ]:
#split car il y a des lignes avec plusieurs codes postaux dans le cadre d'arrondissement par exemple
#df_loyers_m2_exploded = df_loyers_m2.assign(code_postal=df_loyers_m2['code_postal'].str.split('/')).explode('code_postal')

In [ ]:
#mettre les colonnes dans les bons formats
#df_loyers_m2['code_postal'] = df_loyers_m2['code_postal'].fillna(0).astype(str)

#df_annonces_concat['code_postal'] = df_annonces_concat['code_postal'].astype(str)
#df_loyers_m2['code_postal'] = df_loyers_m2['code_postal'].astype(str)
#df_loyers_m2_exploded['code_postal'] = df_loyers_m2_exploded['code_postal'].astype(str)

In [ ]:
df_loyers_m2.to_csv("loyer_moyen_aggreg_clean.csv")

###Merge entre les annonces scrapées et le df prix_m2 pour obtenir le prix de location au m2 par code postal

In [ ]:
df_annonces_loyer = df_annonces_concat.merge(df_loyers_m2, left_on='code_postal', right_on='code_postal', how='left')

In [ ]:
#ajout de 0 au debut du code postal si il n'a que 4 chiffres

df_annonces_loyer['code_postal'] = df_annonces_loyer['code_postal'].apply(lambda x: '0' + str(x) if pd.notna(x) and len(str(x)) == 4 else str(x) if pd.notna(x) else x)

In [ ]:
#vérification des codes postaux à 4 chiffres
nb_codes_a_quatre_chiffres_2 = df_annonces_loyer['code_postal'].str.match(r'^\d{4}$').sum()
nb_codes_a_quatre_chiffres_2

0

In [ ]:
df_annonces_loyer.head(10)

,url,ville,nb_pieces,surface,prix,code_postal,prix_m2,source,departement,departement_nom,region,region_nom,loyer_moyen_par_m2,loyer_borne_basse_par_m2,loyer_borne_haute_par_m2,population
0,https://www.pap.fr/annonces/appartement-vichy-...,Vichy,4,93.0,287000,03200,3086.021505,pap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin,4,110.0,165000,45380,1500.000000,pap,45,LOIRET,24.0,CENTRE,11.668354,9.989444,13.629547,13.0
2,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine,4,74.0,460000,92600,6216.216216,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,22.278354,17.653215,28.115277,81.6
3,https://www.pap.fr/annonces/appartement-cannes...,Cannes,3,82.0,615000,06400,7500.000000,pap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,1,18.0,119000,92220,6611.111111,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,20.597030,16.614022,25.534914,38.5
5,https://www.pap.fr/annonces/appartement-troyes...,Troyes,3,82.0,219000,10000,2670.731707,pap,10,AUBE,44.0,CHAMPAGNE-ARDENNE,12.079487,9.549973,15.278997,61.2
6,https://www.pap.fr/annonces/appartement-paris-...,Paris 3E,2,39.0,470000,75003,12051.282051,pap,75,PARIS,11.0,ILE-DE-FRANCE,33.419452,25.112062,44.475032,35.7
7,https://www.pap.fr/annonces/appartement-marsei...,Marseille 12E,4,81.0,265000,13012,3271.604938,pap,13,BOUCHES-DU-RHONE,93.0,"""PROVENCE-ALPES-COTE DAZUR""",15.790952,12.709815,19.619023,58.7
8,https://www.pap.fr/annonces/appartement-palais...,Palaiseau,5,105.0,325000,91120,3095.238095,pap,91,ESSONNE,11.0,ILE-DE-FRANCE,17.594318,14.624111,21.167783,30.3
9,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,2,28.0,205000,92220,7321.428571,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,20.597030,16.614022,25.534914,38.5


In [ ]:
df_annonces_loyer.isnull().sum()

,0
url,0
ville,0
nb_pieces,0
surface,0
prix,0
code_postal,0
prix_m2,0
source,0
departement,2151
departement_nom,2151


In [ ]:
df_annonces_loyer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8979 entries, 0 to 8978
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       8979 non-null   object 
 1   ville                     8979 non-null   object 
 2   nb_pieces                 8979 non-null   int64  
 3   surface                   8979 non-null   float64
 4   prix                      8979 non-null   int64  
 5   code_postal               8979 non-null   object 
 6   prix_m2                   8979 non-null   float64
 7   source                    8979 non-null   object 
 8   departement               6828 non-null   object 
 9   departement_nom           6828 non-null   object 
 10  region                    6828 non-null   float64
 11  region_nom                6828 non-null   object 
 12  loyer_moyen_par_m2        6828 non-null   float64
 13  loyer_borne_basse_par_m2  6828 non-null   float64
 14  loyer_bo

In [ ]:
df_annonces_loyer.shape

(8979, 16)

In [ ]:
df_annonces_loyer["loyer_mensuel"] = df_annonces_loyer["surface"] * df_annonces_loyer["loyer_moyen_par_m2"]

In [ ]:
df_annonces_loyer.head()

,url,ville,nb_pieces,surface,prix,code_postal,prix_m2,source,departement,departement_nom,region,region_nom,loyer_moyen_par_m2,loyer_borne_basse_par_m2,loyer_borne_haute_par_m2,population,loyer_mensuel
0,https://www.pap.fr/annonces/appartement-vichy-...,Vichy,4,93.0,287000,03200,3086.021505,pap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin,4,110.0,165000,45380,1500.000000,pap,45,LOIRET,24.0,CENTRE,11.668354,9.989444,13.629547,13.0,1283.518985
2,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine,4,74.0,460000,92600,6216.216216,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,22.278354,17.653215,28.115277,81.6,1648.598163
3,https://www.pap.fr/annonces/appartement-cannes...,Cannes,3,82.0,615000,06400,7500.000000,pap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,1,18.0,119000,92220,6611.111111,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,20.597030,16.614022,25.534914,38.5,370.746536


In [ ]:
df_cleaned_loyer = df_annonces_loyer.dropna(subset=['loyer_mensuel'])

In [ ]:
df_cleaned_loyer_problem = df_annonces_loyer[df_annonces_loyer['loyer_mensuel'].isna()]

In [ ]:
df_cleaned_loyer_problem.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2151 entries, 0 to 8972
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       2151 non-null   object 
 1   ville                     2151 non-null   object 
 2   nb_pieces                 2151 non-null   int64  
 3   surface                   2151 non-null   float64
 4   prix                      2151 non-null   int64  
 5   code_postal               2151 non-null   object 
 6   prix_m2                   2151 non-null   float64
 7   source                    2151 non-null   object 
 8   departement               0 non-null      object 
 9   departement_nom           0 non-null      object 
 10  region                    0 non-null      float64
 11  region_nom                0 non-null      object 
 12  loyer_moyen_par_m2        0 non-null      float64
 13  loyer_borne_basse_par_m2  0 non-null      float64
 14  loyer_borne_h

In [ ]:
df_cleaned_loyer.loc[:, "prix_m2_final"] = df_cleaned_loyer["prix"] / df_cleaned_loyer["surface"]

<ipython-input-1618-f31b14cc1408>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_loyer.loc[:, "prix_m2_final"] = df_cleaned_loyer["prix"] / df_cleaned_loyer["surface"]


In [ ]:
df_cleaned_loyer.loc[:, "renta_net"] = (((df_cleaned_loyer["loyer_moyen_par_m2"] * df_cleaned_loyer["surface"]) * 9) / df_cleaned_loyer["prix"])*100

<ipython-input-1619-5c7f2bd2dc1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned_loyer.loc[:, "renta_net"] = (((df_cleaned_loyer["loyer_moyen_par_m2"] * df_cleaned_loyer["surface"]) * 9) / df_cleaned_loyer["prix"])*100


In [ ]:
df_cleaned_loyer.head(10)

,url,ville,nb_pieces,surface,prix,code_postal,prix_m2,source,departement,departement_nom,region,region_nom,loyer_moyen_par_m2,loyer_borne_basse_par_m2,loyer_borne_haute_par_m2,population,loyer_mensuel,prix_m2_final,renta_net
1,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin,4,110.0,165000,45380,1500.000000,pap,45,LOIRET,24.0,CENTRE,11.668354,9.989444,13.629547,13.0,1283.518985,1500.000000,7.001013
2,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine,4,74.0,460000,92600,6216.216216,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,22.278354,17.653215,28.115277,81.6,1648.598163,6216.216216,3.225518
4,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,1,18.0,119000,92220,6611.111111,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,20.597030,16.614022,25.534914,38.5,370.746536,6611.111111,2.803965
5,https://www.pap.fr/annonces/appartement-troyes...,Troyes,3,82.0,219000,10000,2670.731707,pap,10,AUBE,44.0,CHAMPAGNE-ARDENNE,12.079487,9.549973,15.278997,61.2,990.517963,2670.731707,4.070622
6,https://www.pap.fr/annonces/appartement-paris-...,Paris 3E,2,39.0,470000,75003,12051.282051,pap,75,PARIS,11.0,ILE-DE-FRANCE,33.419452,25.112062,44.475032,35.7,1303.358619,12051.282051,2.495793
7,https://www.pap.fr/annonces/appartement-marsei...,Marseille 12E,4,81.0,265000,13012,3271.604938,pap,13,BOUCHES-DU-RHONE,93.0,"""PROVENCE-ALPES-COTE DAZUR""",15.790952,12.709815,19.619023,58.7,1279.067087,3271.604938,4.344001
8,https://www.pap.fr/annonces/appartement-palais...,Palaiseau,5,105.0,325000,91120,3095.238095,pap,91,ESSONNE,11.0,ILE-DE-FRANCE,17.594318,14.624111,21.167783,30.3,1847.403345,3095.238095,5.115886
9,https://www.pap.fr/annonces/appartement-bagneu...,Bagneux,2,28.0,205000,92220,7321.428571,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,20.597030,16.614022,25.534914,38.5,576.716835,7321.428571,2.531928
12,https://www.pap.fr/annonces/appartement-la-cha...,La Chapelle-Saint-Mesmin,4,110.0,165000,45380,1500.000000,pap,45,LOIRET,24.0,CENTRE,11.668354,9.989444,13.629547,13.0,1283.518985,1500.000000,7.001013
13,https://www.pap.fr/annonces/appartement-asnier...,Asnières-Sur-Seine,4,74.0,460000,92600,6216.216216,pap,92,HAUTS-DE-SEINE,11.0,ILE-DE-FRANCE,22.278354,17.653215,28.115277,81.6,1648.598163,6216.216216,3.225518


In [ ]:
df_cleaned_loyer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6828 entries, 1 to 8978
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       6828 non-null   object 
 1   ville                     6828 non-null   object 
 2   nb_pieces                 6828 non-null   int64  
 3   surface                   6828 non-null   float64
 4   prix                      6828 non-null   int64  
 5   code_postal               6828 non-null   object 
 6   prix_m2                   6828 non-null   float64
 7   source                    6828 non-null   object 
 8   departement               6828 non-null   object 
 9   departement_nom           6828 non-null   object 
 10  region                    6828 non-null   float64
 11  region_nom                6828 non-null   object 
 12  loyer_moyen_par_m2        6828 non-null   float64
 13  loyer_borne_basse_par_m2  6828 non-null   float64
 14  loyer_borne_h

In [ ]:
df_cleaned_loyer.to_csv("df_cleaned_loyer.csv")

In [ ]:
df_cleaned_loyer_problem.to_csv("df_cleaned_loyer_problem.csv")

###Top 50 anonces par renta net

In [ ]:
df_cleaned_loyer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6828 entries, 1 to 8978
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       6828 non-null   object 
 1   ville                     6828 non-null   object 
 2   nb_pieces                 6828 non-null   int64  
 3   surface                   6828 non-null   float64
 4   prix                      6828 non-null   int64  
 5   code_postal               6828 non-null   object 
 6   prix_m2                   6828 non-null   float64
 7   source                    6828 non-null   object 
 8   departement               6828 non-null   object 
 9   departement_nom           6828 non-null   object 
 10  region                    6828 non-null   float64
 11  region_nom                6828 non-null   object 
 12  loyer_moyen_par_m2        6828 non-null   float64
 13  loyer_borne_basse_par_m2  6828 non-null   float64
 14  loyer_borne_h

In [ ]:
print(df_cleaned_loyer[df_cleaned_loyer["renta_net"] > 20][["ville", "renta_net"]])


          ville   renta_net
8760  Les Lilas  255.248035
8939      Trets   31.880316
8970      Trets   31.880316


In [ ]:
# filtre sur les valeurs abérante de renta liées à des erreurs de création d'annonces
df_top_renta = df_cleaned_loyer[df_cleaned_loyer["renta_net"] < 15]

#drop duplicates
df_top_renta = df_top_renta.drop_duplicates()

# on garde que le top 50
df_top_renta = df_top_renta.sort_values(by="renta_net", ascending=False).head(50)

In [ ]:
df_top_renta

,url,ville,nb_pieces,surface,prix,code_postal,prix_m2,source,departement,departement_nom,region,region_nom,loyer_moyen_par_m2,loyer_borne_basse_par_m2,loyer_borne_haute_par_m2,population,loyer_mensuel,prix_m2_final,renta_net
6119,https://www.seloger.com/annonces/achat/apparte...,Chancellerie à Bourges,3,58.0,42000,18000,724.0,se_loger,18,CHER,24.0,CENTRE,11.181686,8.117046,15.403400,66.8,648.537792,724.137931,13.897238
7800,https://www.seloger.com/annonces/achat/apparte...,Petite Holande-Lulli à Montbéliard,4,73.0,48000,25200,658.0,se_loger,25,DOUBS,27.0,FRANCHE-COMTE,9.475659,7.593419,11.826378,37.2,691.723101,657.534247,12.969808
4971,https://www.seloger.com/annonces/achat/apparte...,Bessèges,3,70.0,42000,30160,600.0,se_loger,30,GARD,76.0,LANGUEDOC-ROUSSILLON,8.568264,6.954496,10.557301,5.8,599.778480,600.000000,12.852396
8869,https://www.leboncoin.fr/ad/ventes_immobiliere...,Chalonnes-sur-Loire,2,86.0,59248,49290,689.0,bon_coin_ancien,49,MAINE-ET-LOIRE,52.0,PAYS DE LA LOIRE,9.588349,7.531299,12.209723,7.4,824.597995,688.930233,12.525962
5924,https://www.seloger.com/annonces/achat/apparte...,Beausejour à Noyon,3,54.0,48000,60400,889.0,se_loger,60,OISE,32.0,PICARDIE,10.761557,8.634071,13.419274,27.3,581.124094,888.888889,10.896077
5004,https://www.seloger.com/annonces/achat/apparte...,Centre Ville-Les Marins à Châteauroux,2,46.0,42000,36000,913.0,se_loger,36,INDRE,24.0,CENTRE,10.814303,8.527936,13.713652,46.4,497.457925,913.043478,10.659813
8622,https://www.leboncoin.fr/ad/ventes_immobiliere...,Saint-Claude,2,32.0,22000,39200,688.0,bon_coin_ancien,39,JURA,27.0,FRANCHE-COMTE,8.030531,6.180561,10.434236,12.7,256.977004,687.500000,10.512696
8489,https://www.leboncoin.fr/ad/ventes_immobiliere...,Valence,3,60.0,58000,26000,967.0,bon_coin_ancien,26,DROME,84.0,RHONE-ALPES,11.255757,8.937169,14.175863,64.4,675.345448,966.666667,10.479498
6273,https://www.seloger.com/annonces/achat/apparte...,Saint Julien à Le Petit-Quevilly,2,54.0,54600,76140,0.0,se_loger,76,SEINE-MARITIME,28.0,HAUTE-NORMANDIE,11.772503,9.924901,13.964051,21.7,635.715161,1011.111111,10.478821
8668,https://www.leboncoin.fr/ad/ventes_immobiliere...,Échirolles,4,92.0,105000,38130,1141.0,bon_coin_ancien,38,ISERE,84.0,RHONE-ALPES,13.037875,10.319724,16.471970,35.7,1199.484534,1141.304348,10.281296


In [ ]:
df_top_renta.to_csv("top50_renta.csv")

###Ajout des coordonnées GPS lat-long dans le fichier

In [ ]:
import requests
import time

# Charger le fichier CSV
fichier_coordonnees = "top50_renta.csv"
df = pd.read_csv(fichier_coordonnees)

# Ajouter les colonnes pour la latitude et la longitude
if 'lat' not in df.columns:
    df['lat'] = None
if 'long' not in df.columns:
    df['long'] = None

# Fonction pour obtenir les coordonnées GPS à partir du code postal (France)
def get_coordinates(postal_code):
    url = f'https://nominatim.openstreetmap.org/search?format=json&country=France&postalcode={postal_code}'
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code == 200:
        data = response.json()
        if data:
            return float(data[0]['lat']), float(data[0]['lon'])
    return None, None

# Parcourir le DataFrame et remplir les colonnes lat et long
for i, row in df.iterrows():
    if pd.isna(row['lat']) or pd.isna(row['long']):
        lat, lon = get_coordinates(row['code_postal'])
        df.at[i, 'lat'] = lat
        df.at[i, 'long'] = lon
        time.sleep(1)  # Pour éviter d'être bloqué par l'API (max 1 req/sec)

In [ ]:
# Sauvegarder le fichier final avec les nouvelles colonnes
df.to_csv("top_50_coordonnees.csv", index=False)

###Export des top_50 vers Google sheet



In [ ]:
from google.colab import auth
from google.auth import default
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe

# Authentification pour Google Colab
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Chargement du fichier CSV dans un DataFrame Pandas
df = pd.read_csv("top_50_coordonnees.csv")

# Création ou ouverture de la feuille Google Sheets
spreadsheet_name = "top_50_coordonees_sheet"  # Changez le nom si nécessaire
try:
    sh = gc.open(spreadsheet_name)
except gspread.exceptions.SpreadsheetNotFound:
    sh = gc.create(spreadsheet_name)
    sh.share(creds.service_account_email, perm_type='user', role='writer')

# Sélection de la première feuille (ou création si inexistante)
sheet = sh.sheet1

# Envoi des données dans Google Sheets
set_with_dataframe(sheet, df)